<a href="https://colab.research.google.com/github/AnttonLA/BINP37/blob/master/test_bioBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()



Gen RAM Free: 12.7 GB  | Proc size: 159.4 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


# Testing BioBERT

This notebook was created to **test out bioBERT**, see how it worked and confirm that it could be used for my project. I've been mostly using it to run preliminary code. As such, it contains my notes and code I used during the testing process, but not detailed explanation. Often times the code is out or order.

A clean, short bioBERT "manual" can be found in my GitHub account (https://github.com/AnttonLA/BINP37/blob/master/bioBERT.ipynb). That nodebook contains a better description what I lerned here!

**Step 1:** Get the BioBERT repository. Install dependencies

In [1]:
!git clone https://github.com/dmis-lab/biobert.git

Cloning into 'biobert'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 308 (delta 15), reused 8 (delta 4), pack-reused 278
Receiving objects: 100% (308/308), 502.04 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (177/177), done.


In [2]:
!pip install -r biobert/requirements.txt

     |████████████████████████████████| 258.8MB 58kB/s 
     |████████████████████████████████| 11.7MB 31.0MB/s 
     |████████████████████████████████| 573kB 55.7MB/s 
     |████████████████████████████████| 3.0MB 44.3MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.23.0 which is incompatible.
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.11.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.23.0 which is incompatible.
ERROR: google-auth 1.7.2 has requirement setuptools>=40.3.0, but you'll have setupto

**Step 2:**
Unzip the files (dataset + bioBERT weigths) from Drive

In [3]:
!unzip drive/My\ Drive/project_biobert/NERdata.zip

Archive:  drive/My Drive/project_biobert/NERdata.zip
   creating: BC2GM/
  inflating: BC2GM/devel.tsv         
  inflating: BC2GM/test.tsv          
  inflating: BC2GM/train.tsv         
  inflating: BC2GM/train_dev.tsv     
   creating: BC4CHEMD/
  inflating: BC4CHEMD/devel.tsv      
  inflating: BC4CHEMD/test.tsv       
  inflating: BC4CHEMD/train.tsv      
  inflating: BC4CHEMD/train_dev.tsv  
   creating: BC5CDR-chem/
  inflating: BC5CDR-chem/devel.tsv   
  inflating: BC5CDR-chem/test.tsv    
  inflating: BC5CDR-chem/train.tsv   
  inflating: BC5CDR-chem/train_dev.tsv  
   creating: BC5CDR-disease/
  inflating: BC5CDR-disease/devel.tsv  
  inflating: BC5CDR-disease/test.tsv  
  inflating: BC5CDR-disease/train.tsv  
  inflating: BC5CDR-disease/train_dev.tsv  
   creating: JNLPBA/
  inflating: JNLPBA/devel.tsv        
  inflating: JNLPBA/test.tsv         
  inflating: JNLPBA/train.tsv        
  inflating: JNLPBA/train_dev.tsv    
   creating: linnaeus/
  inflating: linnaeus/devel.tsv

In [4]:
!tar -xvf drive/My\ Drive/project_biobert/biobert_v1.1_pubmed.tar

biobert_v1.1_pubmed/
biobert_v1.1_pubmed/model.ckpt-1000000.data-00000-of-00001
biobert_v1.1_pubmed/model.ckpt-1000000.meta
biobert_v1.1_pubmed/bert_config.json
biobert_v1.1_pubmed/vocab.txt
biobert_v1.1_pubmed/model.ckpt-1000000.index


**Step 3:** Create enviromental variables that define where the model, the data and the output go.

Also at this step I'm copying the already fine-tuned model from my drive, but skip that step if you want to fine-tune it yourself!

In [0]:
#Instead of training the model again, copy the trained model from drive!
!cp -r drive/My\ Drive/project_biobert/ner_outputs ./

'!export' is not working correctly for some reason, so I'll set enviromental variables using the python module _os_ instead.

In [0]:
import os
os.environ['BIOBERT_DIR']= './biobert_v1.1_pubmed'
os.environ['NER_DIR'] = './BC5CDR-chem/'
os.environ['OUTPUT_DIR'] = './ner_outputs'

In [0]:
#Not necessary anymore since we are copying the folder
!mkdir -p $OUTPUT_DIR 

Tensor Flow has trouble with cuda versions over 9.0, so we'll have to install version 9.0. **REMEMBER THERE IS A YES/NO QUESTION YOU HAVE TO ANSWER EARLY ON THE INSTALLATION**

This takes a while...

In [7]:
!sudo dpkg -i drive/My\ Drive/project_biobert/cuda-repo-ubuntu1704_9.0.176-1_amd64.deb
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda-9-0

Selecting previously unselected package cuda-repo-ubuntu1704.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../cuda-repo-ubuntu1704_9.0.176-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1704 (9.0.176-1) ...
Setting up cuda-repo-ubuntu1704 (9.0.176-1) ...

Configuration file '/etc/apt/sources.list.d/cuda.list'
 ==> File on system created by you or by a script.
 ==> File also in package provided by package maintainer.
   What would you like to do about it ?  Your options are:
    Y or I  : install the package maintainer's version
    N or O  : keep your currently-installed version
      D     : show the differences between the versions
      Z     : start a shell to examine the situation
 The default action is to keep your current version.
*** cuda.list (Y/I/N/O/D/Z) [default=N] ? y
Installing new version of config file /etc/apt/sources.list.d/cuda.list ...
Executing: /tmp/apt-key-gpghome.FFDgKZimYR/gpg.1.sh --fetch-keys http://developer.downlo

Remember to change the Colab notebook to GPU, otherwise it won't work!

All ready for fine-tuining on NER with default arguments. It takes around 30mins or so.

Note that the results of THE PREDICTION when running 'python run_ner.py' will be recorded as two files: token_test.txt and label_test.txt

In [0]:
!rm -r ner_outputs/

In [0]:
#Fine tunnning. Check number of epochs!
!python biobert/run_ner.py --do_train=true --do_eval=true --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --num_train_epochs=10.0 --data_dir=$NER_DIR --output_dir=$OUTPUT_DIR

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
!cp -r ./ner_outputs ./drive/My\ Drive/project_biobert/BC5CDR_model/20_epochs


cp: cannot create directory './drive/My Drive/project_biobert/BC5CDR_model/20_epochs': No such file or directory


**Prediction/Classification**. THIS DOES NOT WORK

We DO NOT want to use bert-as-service, we want to fine-tune the bioBERT weights we downloaded and then use that for prediction. In this case, for getting the drug-related Name Entities (NE) in inputed text. We want to perform Name  Entity Recognition (NER). This is theoretically possible with bioBERT, however some issues arise when trying. First of all, the code does not seem to be designed to perform predictions once trained (?).

There is an error if both --do_train and --do_eval flags are false. The developers recommend using --do_train=true --do_predict=ture, but with at smaller number of epochs than when fine tuninig. This way, apparently, the training is skipped and only the prediction is carried out. Indeed, we get the messsage:

INFO:tensorflow:Skipping training since max_steps has already saved.


However, other errors appear when doing so. For example, the flag "--init_checkpoint" seems to be necessary to avoid errors!

In [0]:
!python biobert/run_ner.py --help

In [0]:
!head BC5CDR-chem/test.tsv

The	O
Politics	O
of	O
Disease	O
Epidemics	O
:	O
a	O
Comparative	O
Analysis	O
of	O


In [8]:
#Prediction. CHECK NUMBER OF EPOCHS!
!python biobert/run_ner.py --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --data_dir=$NER_DIR --do_train=true --num_train_epochs=1.0 --do_predict=true --output_dir=$OUTPUT_DIR

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

Running 'run_ner.py' with a custom created dataset (e.g., our 100 paper subset) deletes the old "token_test.txt" file (which I imagine is the step it takes before creating the new one), but then fails to create the new one, and complains the file is missing.

Running 'run_ner.py' with the the provided datasets, however, creates the "token_test.txt" file just fine, even if there was none before!

In [0]:
!cp -r drive/My\ Drive/project_biobert/ner_outputs0/token_test.txt ./ner_outputs/ner_outputs0

In [9]:
!head $OUTPUT_DIR/token_test.txt
!cat $OUTPUT_DIR/token_test.txt | wc -l

[CLS]
The
Politics
of
Disease
E
##pid
##em
##ics
:
366627


In [0]:
!head $OUTPUT_DIR/label_test.txt

obtain word level prediction file. This will generate NER_result_conll.txt in ner_outputs.

In [0]:
!python biobert/biocodes/ner_detokenize.py --help

usage: ner_detokenize.py [-h] [--token_test_path TOKEN_TEST_PATH]
                         [--label_test_path LABEL_TEST_PATH]
                         [--answer_path ANSWER_PATH] [--output_dir OUTPUT_DIR]

optional arguments:
  -h, --help            show this help message and exit
  --token_test_path TOKEN_TEST_PATH
  --label_test_path LABEL_TEST_PATH
  --answer_path ANSWER_PATH
  --output_dir OUTPUT_DIR


In [10]:
!python biobert/biocodes/ner_detokenize.py --token_test_path=$OUTPUT_DIR/token_test.txt --label_test_path=$OUTPUT_DIR/label_test.txt --answer_path=$NER_DIR/test.tsv --output_dir=$OUTPUT_DIR

len(bert_pred['toks']):  294927 len(ans['labels']):  585949
## The predicted sentence of BioBERT model looks like trimmed. (The Length of the tokenized input sequence is longer than max_seq_length); Filling O label instead.
   -> Showing 10 words near skipped part : objective of this paper discuss scholars ' narratives on the Bpolitics
## The predicted sentence of BioBERT model looks like trimmed. (The Length of the tokenized input sequence is longer than max_seq_length); Filling O label instead.
   -> Showing 10 words near skipped part : require analysis . [SEP] [CLS] In this paper , epidemics refer
## The predicted sentence of BioBERT model looks like trimmed. (The Length of the tokenized input sequence is longer than max_seq_length); Filling O label instead.
   -> Showing 10 words near skipped part : this paper , politics will also refer to how scholars talked
## The predicted sentence of BioBERT model looks like trimmed. (The Length of the tokenized input sequence is longer than ma

In [11]:
!head $OUTPUT_DIR/NER_result_conll.txt

The O-MISC O-MISC
Politics O-MISC O-MISC
of O-MISC O-MISC
Disease O-MISC O-MISC
Epidemics O-MISC O-MISC
: O-MISC O-MISC
a O-MISC O-MISC
Comparative O-MISC O-MISC
Analysis O-MISC O-MISC
of O-MISC O-MISC


In [0]:
!mkdir ./drive/My\ Drive/project_biobert/BC5CDR_model
!cp -r ./ner_outputs/NER_result_conll.txt ./drive/My\ Drive/project_biobert/BC5CDR_model/


In [0]:
!cat ./s800/test.tsv | wc -l
!head ./s800/test.tsv

entity-level exact match evaluation results:

In [0]:
!perl biobert/biocodes/conlleval.pl < $OUTPUT_DIR/NER_result_conll.txt

processed 114709 tokens with 6241 phrases; found: 7268 phrases; correct: 5141.
accuracy:  95.48%; precision:  70.73%; recall:  82.37%; FB1:  76.11
             MISC: precision:  70.73%; recall:  82.37%; FB1:  76.11  7268


In [0]:
#Get the papers from the subset:
!unzip drive/My\ Drive/project_biobert/comm_use_subset_100.zip

In [0]:
with open('comm_use_subset_100/00352a58c8766861effed18a4b079d1683fec2ec.json') as f:
  data = json.load(f)
  print(json.dumps(data['metadata']['title'], indent = 4, sort_keys=True))

"MINI REVIEW Function of the Deubiquitinating Enzyme USP46 in the Nervous System and Its Regulation by WD40-Repeat Proteins"


In [0]:
#Generate a rudimentary data-set from the titles of the 100 articles

import json
import re
from os import listdir
from os.path import isfile, join

mypath = 'comm_use_subset_100/'
filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
counter = 0
with open('generated_test.tsv', 'w') as out_file:
  for paper in filenames[:95]: #paper 96 gives problems for some reason
    with open(mypath + paper, 'r') as paper_path:
      data = json.load(paper_path)
      title = json.dumps(data['metadata']['title'])
      if title != "": #shitty failsafe
        split_title = title.split() #I need to find a better method to do this...
        for word in split_title:
          word = re.sub(r'\W+', '',word) #For now I'll just strip weird characters
          if word != "": #again shitty failsafe (after the strip some words are empty)
            out_file.write(word + '\t' + 'O' + '\n')
  out_file.write('\n')  # Super important! Prediction won't work without this

!tail generated_test.tsv
!cat generated_test.tsv | wc -l

Myocardium	O
Provide	O
Insight	O
into	O
the	O
Pathobiology	O
of	O
Atrial	O
Fibrillation	O

1277


In [0]:
!rm s800/test.tsv
!head -1284 generated_test.tsv >> s800/test.tsv
!echo "" >> s800/test.tsv #Add newline. CRUCIAL
!tail s800/test.tsv
!cat s800/test.tsv | wc -l

Provide	O
Insight	O
into	O
the	O
Pathobiology	O
of	O
Atrial	O
Fibrillation	O


1278


In [0]:
!python biobert/run_ner.py --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --data_dir=$NER_DIR --do_train=true --num_train_epochs=1.0 --do_predict=true --output_dir=$OUTPUT_DIR

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
!python biobert/biocodes/ner_detokenize.py --token_test_path=$OUTPUT_DIR/token_test.txt --label_test_path=$OUTPUT_DIR/label_test.txt --answer_path=$NER_DIR/test.tsv --output_dir=$OUTPUT_DIR

In [0]:
#Get the NEs recognized by the model!
path = 'ner_outputs/NER_result_conll.txt'
with open(path, 'r') as NER_predicted:
  NE_list = []
  for line in NER_predicted:
    if 'B-MISC' in line or 'I-MISC' in line:
      NE_list.append(line)
print(NE_list)


['HN O-MISC B-MISC\n', 'amino O-MISC B-MISC\n', 'acids O-MISC I-MISC\n', 'amino O-MISC B-MISC\n', 'acids O-MISC I-MISC\n', 'amino O-MISC B-MISC\n', 'acid O-MISC I-MISC\n']
